In [125]:
using CSV, DataFrames, RDatasets, DrWatson
using Flux, Optim
using StatsBase
using Zygote

In [6]:
datapath = datadir("exp_raw");
advertising = CSV.File(joinpath(datapath, "Advertising.csv")) |> DataFrame
advertising = advertising[!,Not(:Column1)]
n,m = size(advertising)
first(advertising,5)
X= [ones(n) standardize(ZScoreTransform,Matrix(advertising[:,Not(:Sales)]), dims=1) ]
y = standardize(ZScoreTransform,advertising.Sales);

# Defining Empirical Risk Function 

In [126]:
ŷ(X,β) = X * β # y_prediction 
loss = L2DistLoss() 
mse(β) = sum(value(loss,ŷ(X,β),y))
mse_i(β) = (i,) -> sum(value(loss,ŷ(X[i,:]',β),y[i])) # to compute empirical risk at each observation 

mse_i (generic function with 1 method)

# Optimizing using Empirical Risk Minimization Framework 

In [127]:
β = zeros(m)
results = optimize(mse,β)

 * Status: success

 * Candidate solution
    Final objective value:     2.045508e+01

 * Found with
    Algorithm:     Nelder-Mead

 * Convergence measures
    √(Σ(yᵢ-ȳ)²)/n ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    145
    f(x) calls:    256


# Extracting the strict local minimizer 

In [234]:
params = Optim.minimizer(results)

4-element Vector{Float64}:
 -8.16027306269304e-7
  0.7530632323952828
  0.5364769753595116
 -0.004324142304595064

In [235]:
norm(gradient(mse,params)[1], Inf)

0.0018993835840421802

### Gradients at each data point - $g_i$

In [132]:
G = Matrix{Float64}(undef,n,m);
for i in 1:n
    G[i,:] = gradient(x->mse_i(x)(i),params)[1]
end

In [136]:
B = G' * G / n 

4×4 Matrix{Float64}:
  0.409102   -0.140573    0.0303325  -0.0316571
 -0.140573    0.739208   -0.26708    -0.0103427
  0.0303325  -0.26708     0.598395    0.160148
 -0.0316571  -0.0103427   0.160148    0.469238

In [143]:
OPG = inv(B) / n 
eigvals(OPG)

4-element Vector{Float64}:
 0.005036217242648921
 0.008646567139358663
 0.014254829451392323
 0.017000355021203825

In [215]:
A = hessian(mse,params) / n 

4×4 Matrix{Float64}:
 2.0          9.17044e-16  6.52811e-16  4.52971e-16
 9.17044e-16  1.99         0.109069     0.112729
 6.52811e-16  0.109069     1.99         0.704666
 4.52971e-16  0.112729     0.704666     1.99

In [216]:
C = inv(A) * B * inv(A)

4×4 Matrix{Float64}:
  0.102275   -0.0354467   0.0133426  -0.0106707
 -0.0354467   0.194618   -0.0903056   0.0116464
  0.0133426  -0.0903056   0.18639    -0.0622513
 -0.0106707   0.0116464  -0.0622513   0.140886

## Checking if both A and B are positive definite 

In [201]:
@show isposdef(A);
@show isposdef(B);

isposdef(A) = true
isposdef(B) = true


# Robust Covariane Matrix using $\hat{C}$

In [236]:
robust_cov = C / n 

4×4 Matrix{Float64}:
  0.000511377  -0.000177234   6.67128e-5   -5.33536e-5
 -0.000177234   0.000973091  -0.000451528   5.82321e-5
  6.67128e-5   -0.000451528   0.000931948  -0.000311257
 -5.33536e-5    5.82321e-5   -0.000311257   0.00070443

# Classical Convariance Matrix using $\hat{A}$

In [219]:
classical_cov = inv(A) / n 

4×4 Matrix{Float64}:
  0.0025       -1.1003e-18   -6.6358e-19   -2.71753e-19
 -1.1003e-18    0.00252415   -0.000100288  -0.000107475
 -6.6358e-19   -0.000100288   0.00287676   -0.00101299
 -2.71753e-19  -0.000107475  -0.00101299    0.00287736

In [213]:
using StatsBase
cov(X)

4×4 Matrix{Float64}:
 0.0  0.0        0.0        0.0
 0.0  1.0        0.0548087  0.0566479
 0.0  0.0548087  1.0        0.354104
 0.0  0.0566479  0.354104   1.0

In [239]:
@show eigvals(C);
@show eigvals(A);
@show eigvals(B);

eigvals(C) = [0.0681588120967666, 0.10031296639954473, 0.1509827824118953, 0.3047145076944646]
eigvals(A) = [1.2853237896526644, 1.9566797166627166, 2.0, 2.7279964936846173]
eigvals(B) = [0.2941115049517326, 0.3507583178774283, 0.5782641734475517, 0.9928086417038937]
